In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import gc
import cv2
import math
import copy
import time
import random
import warnings
import shutil
from pathlib import Path
from PIL import Image
from torchvision import transforms
import cv2

# For data manipulation
import numpy as np
import pandas as pd
from PIL import *

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
!unzip "/content/drive/MyDrive/2023-uou-hd-dt-2-cv.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/0/empty_T7633.png  
  inflating: train/0/empty_T7634.png  
  inflating: train/0/empty_T7635.png  
  inflating: train/0/empty_T7636.png  
  inflating: train/0/empty_T7637.png  
  inflating: train/0/empty_T7638.png  
  inflating: train/0/empty_T7639.png  
  inflating: train/0/empty_T764.png  
  inflating: train/0/empty_T7640.png  
  inflating: train/0/empty_T7641.png  
  inflating: train/0/empty_T7642.png  
  inflating: train/0/empty_T7643.png  
  inflating: train/0/empty_T7644.png  
  inflating: train/0/empty_T7645.png  
  inflating: train/0/empty_T7646.png  
  inflating: train/0/empty_T7647.png  
  inflating: train/0/empty_T7648.png  
  inflating: train/0/empty_T7649.png  
  inflating: train/0/empty_T765.png  
  inflating: train/0/empty_T7650.png  
  inflating: train/0/empty_T7651.png  
  inflating: train/0/empty_T7652.png  
  inflating: train/0/empty_T7653.png  
  inflating: train/0/empty_T7654.png  
  inflating: train/0/empty_T76

In [4]:
!pip install rembg # for library
!pip install rembg[cli]

from rembg import remove

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.7 MB/s eta 0

In [6]:
import tensorflow as tf
import os


input_folder = '/content/test'
output_folder = '/content/test_png'
os.makedirs(output_folder, exist_ok=True)

def convert_jpg_to_png_tf(input_folder, output_folder):
    try:
        file_list = os.listdir(input_folder)

        for file_name in file_list:
            if file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
                input_path = os.path.join(input_folder, file_name)

                try:
                    # 파일 크기 확인
                    file_size = os.path.getsize(input_path)

                    if file_size > 0:  # 파일 크기가 0보다 큰 경우에만 처리
                        # 이미지 읽기
                        image = tf.io.read_file(input_path)
                        image = tf.image.decode_jpeg(image, channels=3)  # channels=3은 RGB 이미지를 의미
                        # 출력 이미지 경로 및 파일 확장자 변경
                        output_file_name = os.path.splitext(file_name)[0] + '.png'
                        output_path = os.path.join(output_folder, output_file_name)
                        # 이미지를 PNG로 저장
                        tf.io.write_file(output_path, tf.image.encode_png(image))
                        print(f'이미지를 성공적으로 저장했습니다. {input_path} → {output_path}')
                    else:
                        print(f'오류: 이미지 파일이 손상되었습니다. {input_path}')

                except tf.errors.InvalidArgumentError:
                    # 손상된 이미지인 경우 건너뛰고 오류 메시지 출력
                    print(f'오류: 이미지 파일이 손상되었습니다. {input_path}')

    except Exception as e:
        print(f'이미지 변환 중 오류가 발생했습니다: {e}')


convert_jpg_to_png_tf(input_folder, output_folder)


이미지를 성공적으로 저장했습니다. /content/test/341.jpg → /content/test_png/341.png
이미지를 성공적으로 저장했습니다. /content/test/1026.jpg → /content/test_png/1026.png
이미지를 성공적으로 저장했습니다. /content/test/267.jpg → /content/test_png/267.png
이미지를 성공적으로 저장했습니다. /content/test/1263.jpg → /content/test_png/1263.png
이미지를 성공적으로 저장했습니다. /content/test/304.jpg → /content/test_png/304.png
이미지를 성공적으로 저장했습니다. /content/test/409.jpg → /content/test_png/409.png
이미지를 성공적으로 저장했습니다. /content/test/196.jpg → /content/test_png/196.png
이미지를 성공적으로 저장했습니다. /content/test/1612.jpg → /content/test_png/1612.png
이미지를 성공적으로 저장했습니다. /content/test/692.jpg → /content/test_png/692.png
이미지를 성공적으로 저장했습니다. /content/test/1816.jpg → /content/test_png/1816.png
이미지를 성공적으로 저장했습니다. /content/test/1272.jpg → /content/test_png/1272.png
이미지를 성공적으로 저장했습니다. /content/test/1332.jpg → /content/test_png/1332.png
이미지를 성공적으로 저장했습니다. /content/test/75.jpg → /content/test_png/75.png
이미지를 성공적으로 저장했습니다. /content/test/1823.jpg → /content/test_png/1823.png
이미지를 성공적으로 저장했습니다. /co

KeyboardInterrupt: ignored

In [ ]:
# train_0 전처리
# 경로 설정
train_path = '/content/train/'
test_path = '/content/test/'



# train 이미지 폴더 경로
empty_folder = "/content/train/0/"
loaded_folder = "/content/train/1/"

# empty_normal
empty_dest = "/content/train/0/"




# reg
empty_reg_dest = "/content/train_reg/0/"

# 폴더가 없다면 생성합니다.
if not os.path.exists(empty_reg_dest):
    os.makedirs(empty_reg_dest)

# 파일 목록을 리스트로
empty_folder_list = os.listdir(empty_folder)
loaded_folder_list = os.listdir(loaded_folder)
print(len(empty_folder_list), len(loaded_folder_list))

    # 빈 수레 이미지 조회
for i in range(len(empty_folder_list)):

    # 이미지를 컬러로 읽기
    image = cv2.imread(empty_folder + empty_folder_list[i])

    """
    이 부분에 전처리를 취하면 전처리가 적용 된 이미지를 반환함
    """



    # 이미지를 reg 로 변환
    output  = remove(image)

    # Normalzed
    zeros_image = np.zeros_like(output)
    normalized_reg_image = cv2.normalize(output, zeros_image, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)

    # rembg inwrite
    cv2.imwrite(empty_reg_dest + empty_folder_list[i], normalized_reg_image)






7748 4623


In [ ]:
# train_1 전처리
# 경로 설정
train_path = '/content/train/'
test_path = '/content/test/'



# train 이미지 폴더 경로
empty_folder = "/content/train/0/"
loaded_folder = "/content/train/1/"

# loaded_normal
loaded_dest = "/content/train/1/"




# reg
loaded_reg_dest = "/content/train_reg/1/"

# 폴더가 없다면 생성합니다.
if not os.path.exists(loaded_reg_dest):
    os.makedirs(loaded_reg_dest)

# 파일 목록을 리스트로
empty_folder_list = os.listdir(empty_folder)
loaded_folder_list = os.listdir(loaded_folder)
print(len(empty_folder_list), len(loaded_folder_list))

    # 빈 수레 이미지 조회
for i in range(len(loaded_folder_list)):

    # 이미지를 컬러로 읽기
    image = cv2.imread(loaded_folder + loaded_folder_list[i])

    """
    이 부분에 전처리를 취하면 전처리가 적용 된 이미지를 반환함
    """



    # 이미지를 reg 로 변환
    output  = remove(image)

    # Normalzed
    zeros_image = np.zeros_like(output)
    normalized_reg_image = cv2.normalize(output, zeros_image, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)

    # rembg inwrite
    cv2.imwrite(loaded_reg_dest + loaded_folder_list[i], normalized_reg_image)